In [1]:
!pip install boto3

     -------------------------------------- 139.2/139.2 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 12.5/12.5 MB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 82.7/82.7 kB 1.5 MB/s eta 0:00:00
     -------------------------------------- 121.4/121.4 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.22
    Uninstalling urllib3-1.22:
      Successfully uninstalled urllib3-1.22


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.28.1 requires urllib3<1.27,>=1.21.1, but you have urllib3 2.2.2 which is incompatible.
tensorboard 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.23.3 which is incompatible.
tensorflow 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.23.3 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import boto3
import json

with open("config.json") as config_file:
    config = json.load(config_file)

s3 = boto3.resource(
    service_name='s3',
    region_name= 'eu-north-1',
    aws_access_key_id = config["AWS_access_key_id"],
    aws_secret_access_key = config["AWS_secret_access_key"]
)

In [5]:
for bucket in s3.buckets.all():
    print(bucket.name)

pii2dataset


In [10]:
import pandas as pd
obj = s3.Bucket('pii2dataset').Object('pii_data.csv').get()
df = pd.read_csv(obj['Body'])
df

,Name,Email,Phone Number,SSN,Credit Card Number,IP Address,Address,Date of Birth
0,Kimberly Stout,kelly00@example.com,695.593.0676x70050,772-68-9340,3580302966034375,202.168.67.82,"38302 Smith Junction New Kirkhaven, KY 18410",1997-09-26
1,Jeanette Scott,barajasdennis@example.com,(758)555-1670x46243,443-22-9091,213151288809043,158.245.89.106,"3757 Ryan Stream Jeffreyfurt, TN 72053",1930-09-03
2,David Brown,steven90@example.com,001-708-641-8838,316-26-0569,2697170525811712,36.111.65.226,"91182 Kirby Greens Suite 148 North Donna, MO 9...",1943-03-20
3,Daniel Velez,ann17@example.com,(563)404-9358x22390,024-86-8887,502010679939,134.15.6.100,"0310 Melissa Loaf Brandonchester, CA 72179",2011-09-16
4,Angela Williams,chadbrown@example.org,(520)242-4090x64452,479-32-8943,3554515070921607,129.95.29.250,"5020 Dixon Glens Monicafurt, KS 53419",1957-11-03
...,...,...,...,...,...,...,...,...
995,Robert Barton,oguerrero@example.org,(537)417-1878,312-30-9761,6011865998515555,89.30.103.7,"445 Kyle Club West Brianmouth, NJ 96672",2006-01-30
996,Amy Hunt,qbrown@example.org,996.291.3690x65088,133-63-0704,3579865065113087,63.18.182.254,"0770 Stephen Forge East Aaronmouth, HI 46344",2012-10-16
997,Lisa Ward,elizabethwilkerson@example.org,256.353.2976,296-55-9571,3540948129318193,209.237.24.213,Unit 4378 Box 8557 DPO AE 52017,1963-04-08
998,Sabrina Espinoza,bethanyjones@example.net,+1-945-422-7719x34073,245-17-3731,4778143662927631,122.35.215.219,"0713 Jessica Freeway Suite 946 Hudsonville, FM...",1948-10-31


### DATA FETCHED FROM AWS S3 CLOUD !

## REGEX PATTERN MATCHING :

In [13]:
import re

# Example regex patterns for labeling PII
patterns = {
    'Name': r'[A-Z][a-z]+\s[A-Z][a-z]+',  # Simplified name pattern
    'Email': r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+',
    'Phone Number': r'(\+?\d{1,3})?[\s.-]?\(?\d{1,4}\)?[\s.-]?\d{1,4}[\s.-]?\d{1,9}',
    'SSN': r'\d{3}-\d{2}-\d{4}',
    'Credit Card Number': r'\b(?:\d[ -]*?){13,16}\b',
    'IP Address': r'\b(?:\d{1,3}\.){3}\d{1,3}\b',
    'Address': r'[0-9]{1,5}\s\w+\s\w+',  # Simplified address pattern
    'Date of Birth': r'\b\d{4}-\d{2}-\d{2}\b'
}

# Function to label PII in the dataset
def label_pii(text, patterns):
    for pii_type, pattern in patterns.items():
        if re.search(pattern, text):
            return pii_type
    return 'Non-PII'

# Apply the labeling to the dataset
df['Label'] = df.apply(lambda row: label_pii(' '.join(row.astype(str)), patterns), axis=1)


In [16]:
from sklearn.preprocessing import LabelEncoder
# Label the data
def label_pii(text):
    if "@" in text:  # Simplified PII detection
        return 'Email'
    elif "-" in text and len(text) == 11:  # SSN pattern
        return 'SSN'
    elif len(text) == 16 and text.isdigit():  # Credit Card pattern
        return 'Credit Card'
    elif text.count(".") == 3:  # IP Address pattern
        return 'IP Address'
    elif any(char.isdigit() for char in text):  # Phone numbers, addresses, etc.
        return 'Other PII'
    else:
        return 'Non-PII'

df['Label2'] = df.apply(lambda row: label_pii(' '.join(row.astype(str))), axis=1)

# Combine text columns for training
#df['text_data'] = pii_dataset.apply(lambda row: ' '.join(row.astype(str)), axis=1)

# Encode the labels
label_encoder = LabelEncoder()
df['Label_encoded'] = label_encoder.fit_transform(df['Label2'])

In [17]:
df

,Name,Email,Phone Number,SSN,Credit Card Number,IP Address,Address,Date of Birth,Label,Label2,Label_encoded
0,Kimberly Stout,kelly00@example.com,695.593.0676x70050,772-68-9340,3580302966034375,202.168.67.82,"38302 Smith Junction New Kirkhaven, KY 18410",1997-09-26,Name,Email,0
1,Jeanette Scott,barajasdennis@example.com,(758)555-1670x46243,443-22-9091,213151288809043,158.245.89.106,"3757 Ryan Stream Jeffreyfurt, TN 72053",1930-09-03,Name,Email,0
2,David Brown,steven90@example.com,001-708-641-8838,316-26-0569,2697170525811712,36.111.65.226,"91182 Kirby Greens Suite 148 North Donna, MO 9...",1943-03-20,Name,Email,0
3,Daniel Velez,ann17@example.com,(563)404-9358x22390,024-86-8887,502010679939,134.15.6.100,"0310 Melissa Loaf Brandonchester, CA 72179",2011-09-16,Name,Email,0
4,Angela Williams,chadbrown@example.org,(520)242-4090x64452,479-32-8943,3554515070921607,129.95.29.250,"5020 Dixon Glens Monicafurt, KS 53419",1957-11-03,Name,Email,0
...,...,...,...,...,...,...,...,...,...,...,...
995,Robert Barton,oguerrero@example.org,(537)417-1878,312-30-9761,6011865998515555,89.30.103.7,"445 Kyle Club West Brianmouth, NJ 96672",2006-01-30,Name,Email,0
996,Amy Hunt,qbrown@example.org,996.291.3690x65088,133-63-0704,3579865065113087,63.18.182.254,"0770 Stephen Forge East Aaronmouth, HI 46344",2012-10-16,Name,Email,0
997,Lisa Ward,elizabethwilkerson@example.org,256.353.2976,296-55-9571,3540948129318193,209.237.24.213,Unit 4378 Box 8557 DPO AE 52017,1963-04-08,Name,Email,0
998,Sabrina Espinoza,bethanyjones@example.net,+1-945-422-7719x34073,245-17-3731,4778143662927631,122.35.215.219,"0713 Jessica Freeway Suite 946 Hudsonville, FM...",1948-10-31,Name,Email,0
